In [1]:
kernel.silent(true)

In [2]:
import coursierapi.MavenRepository
interp.repositories() ++= Seq(MavenRepository.of("https://jitpack.io"))

In [47]:
import $ivy.`com.github.propi:rdfrules:1.5.0`
import collection._
import org.apache.jena.riot.Lang

import scala.util.control.Breaks._
import scala.collection.immutable.ListMap

import com.github.propi.rdfrules.data._
import com.github.propi.rdfrules.algorithm.amie._
import com.github.propi.rdfrules.algorithm.dbscan._
import com.github.propi.rdfrules.utils._
import com.github.propi.rdfrules.index._
import com.github.propi.rdfrules.rule._
import com.github.propi.rdfrules.ruleset._
kernel.silent(false)

import $ivy.$                                

import collection._

import org.apache.jena.riot.Lang


import scala.util.control.Breaks._

import scala.collection.immutable.ListMap


import com.github.propi.rdfrules.data._

import com.github.propi.rdfrules.algorithm.amie._

import com.github.propi.rdfrules.algorithm.dbscan._

import com.github.propi.rdfrules.utils._

import com.github.propi.rdfrules.index._

import com.github.propi.rdfrules.rule._

import com.github.propi.rdfrules.ruleset._


In [4]:
val jaurDataset = Dataset.fromCache("../cache/jaurCube.cache")
jaurDataset.size

jaurDataset: Dataset = com.github.propi.rdfrules.data.Dataset@1812c95a
res3_1: Int = 56871

In [5]:
val yagoHop0 = Dataset.fromCache("../cache/yago-hop0.cache")
yagoHop0.size

yagoHop0: Dataset = com.github.propi.rdfrules.data.Dataset@7e6e7673
res4_1: Int = 49375

In [6]:
val yagoHop1 = Dataset.fromCache("../cache/yago-hop1.cache")
yagoHop1.size

yagoHop1: Dataset = com.github.propi.rdfrules.data.Dataset@40e35c90
res5_1: Int = 1208813

In [7]:
val yagoHop2 = Dataset.fromCache("../cache/yago-hop2.cache")
yagoHop2.size

yagoHop2: Dataset = com.github.propi.rdfrules.data.Dataset@2bdf43b
res6_1: Int = 3573385

In [8]:
val yagoDataset = yagoHop0 + yagoHop1 + yagoHop2
yagoDataset.size

yagoDataset: Dataset = com.github.propi.rdfrules.data.Dataset@6b2fa34d
res7_1: Int = 4831573

In [9]:
val rdfsLabel = "http://www.w3.org/2000/01/rdf-schema#label"
val rdfsComment = "http://www.w3.org/2000/01/rdf-schema#comment"
val alternateName = "http://schema.org/alternateName"
val image = "http://schema.org/image"
val rdfType = "http://www.w3.org/1999/02/22-rdf-syntax-ns#type"

val rounded = (value: Double, scale: Integer) => BigDecimal(value).setScale(scale, BigDecimal.RoundingMode.HALF_UP).toDouble

val yagoDatasetFiltered = yagoDataset.
filter(q => !q.triple.predicate.hasSameUriAs(rdfsLabel) &&
                !q.triple.predicate.hasSameUriAs(rdfsComment) &&
                !q.triple.predicate.hasSameUriAs(alternateName) &&
                !q.triple.predicate.hasSameUriAs(rdfType) &&
                !q.triple.predicate.hasSameUriAs(image))

val ratio: Double = (yagoDatasetFiltered.size.toDouble / yagoDataset.size.toDouble)
yagoDatasetFiltered.size + " / " + yagoDataset.size + " = " + rounded(ratio,2)*100 + "%"

rdfsLabel: String = "http://www.w3.org/2000/01/rdf-schema#label"
rdfsComment: String = "http://www.w3.org/2000/01/rdf-schema#comment"
alternateName: String = "http://schema.org/alternateName"
image: String = "http://schema.org/image"
rdfType: String = "http://www.w3.org/1999/02/22-rdf-syntax-ns#type"
rounded: (Double, Integer) => Double = ammonite.$sess.cmd8$Helper$$Lambda$2898/0x0000000840cbb040@15888007
yagoDatasetFiltered: Dataset = com.github.propi.rdfrules.data.Dataset@5d8da85a
ratio: Double = 0.37625551761300097
res8_8: String = "1817906 / 4831573 = 38.0%"

In [10]:
// ref area linking yago csu
val refAreaLinking = Dataset("../data/linking/yagoCZSOLinking.ttl")
refAreaLinking.size

refAreaLinking: Dataset = com.github.propi.rdfrules.data.Dataset@7e130660
res9_1: Int = 90

In [11]:
val dataset = jaurDataset + yagoDatasetFiltered + refAreaLinking
dataset.size

dataset: Dataset = com.github.propi.rdfrules.data.Dataset@550c2ba8
res10_1: Int = 1874867

In [12]:
val index = dataset.index()
index.cache("../cache/yaurYagoIndex.cache")

2021-05-06 16:38:22:844 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Predicates trimming.
2021-05-06 16:38:24:203 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Subjects indexing.
2021-05-06 16:38:26:156 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Subjects trimming.
2021-05-06 16:38:26:405 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Objects indexing.
2021-05-06 16:38:27:500 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Objects trimming.


index: Index = com.github.propi.rdfrules.index.Index$FromDatasetFullyPreservedIndex@e1286c
res11_1: Index = com.github.propi.rdfrules.index.Index$FromDatasetFullyPreservedIndex@e1286c

In [13]:
val uri = (value: String) => TripleItem.Uri(value)
val qbDataSet = uri("http://purl.org/linked-data/cube#dataSet")
// in all cubes
val czsoUri = "http://data.czso.cz/ontology/"
val unemploymentRate = uri(czsoUri+"podilNezamestnanych")
val reachableApplicants = uri(czsoUri+"dosazitelniNeumisteniUchazeciOZamestnani")
// only in total cubes
val unplacedApplicants = uri(czsoUri+"neumisteniUchazeciOZamestnani")
val vacaniesCount = uri(czsoUri+"pocetVolnychMist")
val measures = Array(unemploymentRate,reachableApplicants,unplacedApplicants,vacaniesCount)

val oneOfMeasures = OneOf(unemploymentRate,reachableApplicants,unplacedApplicants,vacaniesCount)
val refArea = uri("http://data.czso.cz/ontology/refArea")
val constantsAtObject = RuleConstraint.ConstantsAtPosition.ConstantsPosition.Object
val constantsOnlyAtObject = RuleConstraint.ConstantsAtPosition(constantsAtObject)

uri: String => TripleItem.Uri = ammonite.$sess.cmd12$Helper$$Lambda$3092/0x0000000840e38840@6001e42f
qbDataSet: TripleItem.Uri = LongUri("http://purl.org/linked-data/cube#dataSet")
czsoUri: String = "http://data.czso.cz/ontology/"
unemploymentRate: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/podilNezamestnanych"
)
reachableApplicants: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/dosazitelniNeumisteniUchazeciOZamestnani"
)
unplacedApplicants: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/neumisteniUchazeciOZamestnani"
)
vacaniesCount: TripleItem.Uri = LongUri(
  "http://data.czso.cz/ontology/pocetVolnychMist"
)
measures: Array[TripleItem.Uri] = Array(
  LongUri("http://data.czso.cz/ontology/podilNezamestnanych"),
  LongUri(
    "http://data.czso.cz/ontology/dosazitelniNeumisteniUchazeciOZamestnani"
  ),
  LongUri("http://data.czso.cz/ontology/neumisteniUchazeciOZamestnani"),
  LongUri("http://data.czso.cz/ontology/pocetVolnychMist")
)
oneOfMeasures

In [14]:
val oneMeasurePattern = (
    AtomPattern(subject = 'b', graph = uri("yago")) &:
    AtomPattern(subject = 'a', predicate = refArea, `object` = 'b', graph = uri("czso")) &:
    AtomPattern(subject = 'a', predicate = qbDataSet, graph = uri("czso"))
    =>: 
    AtomPattern(subject = 'a', predicate = oneOfMeasures, graph = uri("czso"))
)

oneMeasurePattern: RulePattern = RulePattern(
  Vector(
    AtomPattern(Variable(Variable(1)), Any, Any, Constant(LongUri("yago"))),
    AtomPattern(
      Variable(Variable(0)),
      Constant(LongUri("http://data.czso.cz/ontology/refArea")),
      Variable(Variable(1)),
      Constant(LongUri("czso"))
    ),
    AtomPattern(
      Variable(Variable(0)),
      Constant(LongUri("http://purl.org/linked-data/cube#dataSet")),
      Any,
      Constant(LongUri("czso"))
    )
  ),
  Some(
    AtomPattern(
      Variable(Variable(0)),
      OneOf(
        ArrayBuffer(
          Constant(LongUri("http://data.czso.cz/ontology/podilNezamestnanych")),
          Constant(
            LongUri(
              "http://data.czso.cz/ontology/dosazitelniNeumisteniUchazeciOZamestnani"
            )
          ),
          Constant(
            LongUri(
              "http://data.czso.cz/ontology/neumisteniUchazeciOZamestnani"
            )
          ),
          Constant(LongUri("http://data.czso.cz/ontol

In [15]:
val oneMeasureTask = Amie()
    .addThreshold(Threshold.MinSupport(30))
    .addThreshold(Threshold.MaxRuleLength(6))
    .addConstraint(constantsOnlyAtObject)
    .addPattern(oneMeasurePattern)

oneMeasureTask: com.github.propi.rdfrules.algorithm.RulesMining = com.github.propi.rdfrules.algorithm.amie.Amie@25fee45f

In [16]:
val startTimeMillis = System.currentTimeMillis()
val oneMeasureTaskRuleset = index.mine(oneMeasureTask)
println("rules: "+oneMeasureTaskRuleset.size)
val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
println("duration: " + durationSeconds + "s")

2021-05-06 16:38:30:762 +0200 [scala-interpreter-1] INFO com.github.propi.rdfrules.utils.Debugger - Amie task settings:
MinHeadSize=100,
MinHeadCoverage=0.0,
MinSupport=30,
MaxThreads=4,
MinAtomSize=0,
MaxRuleLength=6,
WithConstants=true,
ConstantsPosition=Object,
Timeout=-1,
WithDuplicitPredicates=true,
Patterns=List(Mapped(Vector(Mapped(Variable(?b),Any,Any,Constant(Constant(-1596161386))), Mapped(Variable(?a),Constant(Constant(1481837794)),Variable(?b),Constant(Constant(1015601977))), Mapped(Variable(?a),Constant(Constant(624690160)),Any,Constant(Constant(1015601977)))),Some(Mapped(Variable(?a),OneOf(ArrayBuffer(Constant(Constant(-2070273298)), Constant(Constant(1659106226)), Constant(Constant(1797717682)), Constant(Constant(1142069620)))),Any,Constant(Constant(1015601977)))),false,false)),
OnlyPredicates=None,
WithoutPredicates=None


rules: 312917
duration: 184s


startTimeMillis: Long = 1620311910731L
oneMeasureTaskRuleset: Ruleset = com.github.propi.rdfrules.ruleset.Ruleset@24974fdb
endTimeMillis: Long = 1620312095119L
durationSeconds: Long = 184L

In [17]:
oneMeasureTaskRuleset
.export("../rulesets/jaurYagoOneMeasureTaskRuleset.txt")

In [19]:
val startTimeMillis = System.currentTimeMillis()
val oneMeasureTaskRulesetFiltered = oneMeasureTaskRuleset
.filterResolved(rr => {rr.body.count(i => measures.contains(i.predicate)) == 0})
.computePcaConfidence(0.6)
.sortBy(Measure.PcaConfidence, Measure.Support)
.cache
oneMeasureTaskRulesetFiltered.export("../rulesets/jaurYagoOneMeasureTaskRulesetFiltered.txt")
println("rules: " + oneMeasureTaskRulesetFiltered.size)
val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
println("duration: " + durationSeconds + "s")

rules: 2236
duration: 118s


startTimeMillis: Long = 1620312157934L
oneMeasureTaskRulesetFiltered: Ruleset = com.github.propi.rdfrules.ruleset.Ruleset@3ab3c32
endTimeMillis: Long = 1620312275937L
durationSeconds: Long = 118L

In [20]:
// clustering
val oneMeasureTaskRulesetFilteredClustered = oneMeasureTaskRulesetFiltered
.makeClusters {
implicit val ruleSimilarityCounting: SimilarityCounting[Rule.Simple] = SimilarityCounting.AtomsSimilarityCounting
DbScan(minNeighbours = 3, minSimilarity = 0.85)
}
.cache

oneMeasureTaskRulesetFilteredClustered: Ruleset = com.github.propi.rdfrules.ruleset.Ruleset@61283ee5

In [21]:
val startTimeMillis = System.currentTimeMillis()
oneMeasureTaskRulesetFilteredClustered
.cache("../cache/oneMeasureTaskRulesetFilteredClustered.cache")
.export("../rulesets/jaurYagoOneMeasureTaskRulesetFilteredClustered.txt")
val endTimeMillis = System.currentTimeMillis()
val durationSeconds = (endTimeMillis - startTimeMillis) / 1000
println("duration: " + durationSeconds + "s")

duration: 0s


startTimeMillis: Long = 1620313449272L
endTimeMillis: Long = 1620313449309L
durationSeconds: Long = 0L

In [4]:
val index = Index.fromCache("../cache/yaurYagoIndex.cache", false)
val oneMeasureTaskRulesetFilteredClustered = Ruleset.fromCache(index, "../cache/oneMeasureTaskRulesetFilteredClustered.cache")

index: Index = com.github.propi.rdfrules.index.Index$FromCacheFullyPreservedIndex@6df7b072
oneMeasureTaskRulesetFilteredClustered: Ruleset = com.github.propi.rdfrules.ruleset.Ruleset@7d3078b1

In [34]:
val filterByCluster = (r: ResolvedRule, cluster: Int) => 
r.measures.get(Measure.Cluster).get == Measure.Cluster(cluster)

filterByCluster: (ResolvedRule, Int) => Boolean = ammonite.$sess.cmd33$Helper$$Lambda$3441/0x0000000840de0840@44e48066

In [48]:
var clustersMap: ListMap[Int, Int] = ListMap()
breakable {
    var counter = 0
    while (true) {
        val size = oneMeasureTaskRulesetFilteredClustered.filterResolved(r => filterByCluster(r,counter)).size
        if (size > 0) {
            clustersMap = clustersMap + (counter -> size)
            counter = counter + 1
        } else break;
    }
}
clustersMap = ListMap(clustersMap.toSeq.sortWith(_._2 > _._2):_*)

clustersMap: ListMap[Int, Int] = ListMap(
  103 -> 388,
  88 -> 327,
  98 -> 211,
  92 -> 178,
  100 -> 126,
  48 -> 113,
  78 -> 109,
  4 -> 72,
  68 -> 59,
  56 -> 50,
  101 -> 46,
  95 -> 44,
  43 -> 39,
  39 -> 33,
  41 -> 32,
  87 -> 25,
  44 -> 24,
  37 -> 23,
  38 -> 23,
  52 -> 21,
  55 -> 20,
  58 -> 16,
  102 -> 15,
  91 -> 14,
  85 -> 13,
  46 -> 11,
  45 -> 10,
  29 -> 9,
  54 -> 9,
  21 -> 8,
  51 -> 8,
  53 -> 8,
  67 -> 8,
  96 -> 8,
  47 -> 7,
  32 -> 6,
  33 -> 6,
  59 -> 6,
...